In [ ]:
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Reshape, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.losses import categorical_crossentropy
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = np.load('face_data.npy')
print(data[:5])
np.random.shuffle(data)
print(data[:5])
X = data[:, 1:]
Y = data[:,0]

[['kartik' '192' '192' ... '183' '183' '183']
 ['kartik' '189' '189' ... '189' '189' '189']
 ['kartik' '188' '188' ... '76' '74' '73']
 ['kartik' '46' '46' ... '175' '175' '175']
 ['kartik' '188' '188' ... '63' '65' '66']]
[['gurprasad' '196' '196' ... '50' '49' '49']
 ['gurprasad' '204' '204' ... '49' '50' '50']
 ['gurprasad' '202' '202' ... '58' '61' '62']
 ['gurprasad' '206' '206' ... '50' '50' '50']
 ['kartik' '188' '188' ... '63' '65' '66']]


In [ ]:
X = X.reshape(X.shape[0], 300, 300)

In [ ]:
X.shape

(20, 300, 300)

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(Y)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)
print(y)

[0 0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 1 1]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
y.shape

(20, 2)

In [ ]:
X, y = X.astype(np.uint8), y.astype(np.uint8)

In [ ]:
in_layer = Input(shape = (300,300))
reshape = Reshape((300, 300, 1))(in_layer)
c1 = Conv2D(32, (2,2))(reshape)
p1 = MaxPooling2D((2, 2))(c1)
c2 = Conv2D(64, (2,2))(p1)
p2 = MaxPooling2D((2, 2))(c2)
c3 = Conv2D(128, (2,2))(p2)
p3 = MaxPooling2D((2, 2))(c3)
drp1 = Dropout(0.2)(p3)
flat = Flatten()(drp1)
d1 = Dense(1024, activation="tanh")(flat)
d2 = Dense(512, activation="tanh")(d1)
drp2 = Dropout(0.5)(d2)
d3 = Dense(300, activation="tanh")(drp2)
drp3 = Dropout(0.5)(d3)
out = Dense(2, activation="softmax")(drp3)

In [ ]:
model = Model(in_layer, out)

In [ ]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 300, 300, 1)]     0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 299, 299, 32)      160       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 148, 148, 64)      8256      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 73, 73, 128)       32896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 128)       0   

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(X, y, epochs=10, steps_per_epoch=10)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
path = "/content/gdrive/My Drive/facemodel.hdf5"
model.save(path, save_format="hdf5")